## Deliverable 2. Create a Customer Travel Destinations Map.

In [7]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np

# Import API key
from config_api import gmaps_key as gkey

# Configure gmaps API key
gmaps.configure(api_key=gkey)

In [8]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Weather
0,0,Saint George,US,2021-07-25 20:15:14,37.1041,-113.5841,96.80,34,1,1.01,clear sky
1,1,General Cepeda,MX,2021-07-25 20:15:15,25.3833,-101.4500,85.41,21,2,15.12,clear sky
2,2,Mataura,NZ,2021-07-25 20:15:15,-46.1927,168.8643,46.47,92,100,3.38,overcast clouds
3,3,Shahrud,IR,2021-07-25 20:15:15,36.4182,54.9763,77.61,52,0,10.18,clear sky
4,4,Mitu,CO,2021-07-25 20:15:15,1.1983,-70.1733,77.32,80,81,3.38,broken clouds


In [9]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input('Enter minimum temperature for trip'))
max_temp = float(input('Enter maximum temperature for trip'))

In [10]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.

custom_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & 
                                    (city_data_df["Max Temp"] >= min_temp)]
custom_cities_df.head(10)

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Weather
0,0,Saint George,US,2021-07-25 20:15:14,37.1041,-113.5841,96.80,34,1,1.01,clear sky
1,1,General Cepeda,MX,2021-07-25 20:15:15,25.3833,-101.4500,85.41,21,2,15.12,clear sky
2,2,Mataura,NZ,2021-07-25 20:15:15,-46.1927,168.8643,46.47,92,100,3.38,overcast clouds
3,3,Shahrud,IR,2021-07-25 20:15:15,36.4182,54.9763,77.61,52,0,10.18,clear sky
4,4,Mitu,CO,2021-07-25 20:15:15,1.1983,-70.1733,77.32,80,81,3.38,broken clouds
6,6,Castro,BR,2021-07-25 20:13:11,-24.7911,-50.0119,65.03,61,2,3.85,clear sky
7,7,Ilulissat,GL,2021-07-25 20:15:16,69.2167,-51.1000,53.62,37,100,9.22,overcast clouds
8,8,Tuatapere,NZ,2021-07-25 20:15:16,-46.1333,167.6833,46.81,95,100,4.74,overcast clouds
9,9,Bredasdorp,ZA,2021-07-25 20:15:16,-34.5322,20.0403,52.32,51,100,4.68,overcast clouds
10,10,Montrose,US,2021-07-25 20:15:17,38.4166,-108.2673,76.41,54,92,6.71,overcast clouds


In [11]:
# 4a.Determine if there are any empty rows.
custom_cities_df.count()

City_ID            646
City               646
Country            641
Date               646
Lat                646
Lng                646
Max Temp           646
Humidity           646
Cloudiness         646
Wind Speed         646
Current Weather    646
dtype: int64

In [12]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_cities_df = custom_cities_df.dropna()

In [13]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_cities_df[["City", "Country", "Max Temp", "Lat", "Lng",'Current Weather']].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Current Weather,Hotel Name
0,Saint George,US,96.80,37.1041,-113.5841,clear sky,
1,General Cepeda,MX,85.41,25.3833,-101.4500,clear sky,
2,Mataura,NZ,46.47,-46.1927,168.8643,overcast clouds,
3,Shahrud,IR,77.61,36.4182,54.9763,clear sky,
4,Mitu,CO,77.32,1.1983,-70.1733,broken clouds,
6,Castro,BR,65.03,-24.7911,-50.0119,clear sky,
7,Ilulissat,GL,53.62,69.2167,-51.1000,overcast clouds,
8,Tuatapere,NZ,46.81,-46.1333,167.6833,overcast clouds,
9,Bredasdorp,ZA,52.32,-34.5322,20.0403,overcast clouds,
10,Montrose,US,76.41,38.4166,-108.2673,overcast clouds,


In [14]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": gkey
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
        
    except (IndexError):
        print("Cannot find hotel")


        

Cannot find hotel
Cannot find hotel
Cannot find hotel
Cannot find hotel
Cannot find hotel
Cannot find hotel
Cannot find hotel
Cannot find hotel
Cannot find hotel
Cannot find hotel
Cannot find hotel
Cannot find hotel
Cannot find hotel
Cannot find hotel
Cannot find hotel
Cannot find hotel
Cannot find hotel
Cannot find hotel
Cannot find hotel
Cannot find hotel
Cannot find hotel
Cannot find hotel
Cannot find hotel
Cannot find hotel
Cannot find hotel
Cannot find hotel
Cannot find hotel
Cannot find hotel
Cannot find hotel
Cannot find hotel
Cannot find hotel
Cannot find hotel
Cannot find hotel
Cannot find hotel
Cannot find hotel
Cannot find hotel
Cannot find hotel
Cannot find hotel
Cannot find hotel
Cannot find hotel
Cannot find hotel
Cannot find hotel
Cannot find hotel
Cannot find hotel
Cannot find hotel
Cannot find hotel
Cannot find hotel
Cannot find hotel
Cannot find hotel
Cannot find hotel
Cannot find hotel
Cannot find hotel
Cannot find hotel
Cannot find hotel
Cannot find hotel
Cannot fin

In [ ]:
hotel_df

,City,Country,Max Temp,Lat,Lng,Current Weather,Hotel Name
1,Punta Arenas,CL,41.11,-53.1500,-70.9167,broken clouds,Hotel Dreams Del Estrecho
2,Chaoyang,CN,74.28,41.5703,120.4586,broken clouds,Chaoyang Genting International Hotel
3,Saint-Philippe,RE,67.59,-21.3585,55.7679,light rain,"Chambres d'hôte ""La Trinité"""
4,Albany,US,81.25,42.6001,-73.9662,overcast clouds,
5,Rikitea,PF,69.26,-23.1203,-134.9692,clear sky,People ThankYou
...,...,...,...,...,...,...,...
685,Banjarmasin,ID,69.89,-3.3244,114.5910,few clouds,Swiss-Belhotel Borneo Banjarmasin
686,Jilotepec,MX,72.34,19.6000,-96.9333,broken clouds,Auto-Hotel Mediterráneo
687,Pula,HR,76.06,44.8683,13.8481,clear sky,Park Plaza Histria Pula
689,Banda Aceh,ID,80.37,5.5577,95.3222,overcast clouds,Sulthan Hotel International


In [16]:
# 7. Drop the rows where there is no Hotel Name.

hotel_df['Hotel Name'].replace('', np.nan, inplace=True)
hotel_df.dropna(subset=['Hotel Name'], inplace=True)
hotel_df

,City,Country,Max Temp,Lat,Lng,Current Weather,Hotel Name
0,Saint George,US,96.80,37.1041,-113.5841,clear sky,Best Western Plus Abbey Inn
1,General Cepeda,MX,85.41,25.3833,-101.4500,clear sky,Hacienda Los Jose
2,Mataura,NZ,46.47,-46.1927,168.8643,overcast clouds,Ellie's Villa
3,Shahrud,IR,77.61,36.4182,54.9763,clear sky,Royal Hotel
6,Castro,BR,65.03,-24.7911,-50.0119,clear sky,CHACARA BAILLY
...,...,...,...,...,...,...,...
656,Lakatoro,VU,78.08,-16.0999,167.4164,light rain,Lakatoro Palm Lodge
657,Bud,US,88.21,39.4470,-86.1758,scattered clouds,Ted McQuinn Trust
658,Katsuura,JP,80.11,35.1333,140.3000,broken clouds,Katsuura Hotel Mikazuki
659,Kudahuvadhoo,MV,81.28,2.6708,72.8944,broken clouds,Niyama Private Islands Maldives


In [ ]:
# 8a. Create the output File (CSV)
output_data_file= "../weather_data/WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [21]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt><b>Hotel Name</b></dt><dd>{Hotel Name}</dd>
<dt><b>City</dt></b><dd>{City}</dd>
<dt><b>Country</dt></b><dd>{Country}</dd>
<dt><b>Max Temp</dt></b><dd>{Max Temp} °F</dd> 
<dt><b>Current Weather</b></dt><dd>{Current Weather}</dd>  
</dl>
"""


#Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = (info_box_template.format(**row) for index, row in hotel_df.iterrows())

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [22]:
# Add a marker layer for each city to the map. 

hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

# Call the figure to plot the data.
fig = gmaps.figure(center=(38.02077, 23.73524), zoom_level=1.8)
fig.add_layer(marker_layer)


# 11b. Display the figure
fig




Figure(layout=FigureLayout(height='420px'))